In [ ]:
import os 
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import pandas as pd

plt.ion()

In [ ]:
outline = gpd.read_file(os.path.abspath('data_files/NI_outline.shp'))
towns = gpd.read_file(os.path.abspath('data_files/Towns.shp'))
counties = gpd.read_file(os.path.abspath('data_files/Counties.shp'))
water = gpd.read_file(os.path.abspath('data_files/Water.shp'))
aonb = gpd.read_file(os.path.abspath('data_files/AONB.shp'))
assi = gpd.read_file(os.path.abspath('data_files/ASSI.shp'))

lepi = pd.read_csv('data_files/egm722_project_data_NBN.xlsx')
lepi.head()